In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
imageio.plugins.freeimage.download()
import os

%matplotlib widget

In [ ]:
def inds_to_edges(V, edges):
    return V[edges.flatten()].reshape((*edges.shape, -1))

def get_axis_2D():
    fig = plt.figure(figsize=(8,6))
    return plt.axes()

def get_axis_3D():
    fig = plt.figure(figsize=(8,6))
    return plt.axes(projection='3d')

def vis_transform(V, edges, T, ax=None, invisibles=None, plot_lines=True):
    if ax is None:
        if V.shape[-1] == 3:
            ax = get_axis_2D()
        else:
            ax = get_axis_3D()
    
    V_T = np.matmul(V, T)
    
    E = inds_to_edges(V, edges)
    E_T = inds_to_edges(V_T, edges)
    
    if not invisibles:
        _min = np.min((V, V_T), axis=(0,1))[:3]
        _max = _min + (np.max((V, V_T), axis=(0,1))[:3] - _min).max()
    
        ax.plot(*np.array((_min, _max)).T, linewidth=0)
    else:
        ax.plot(*np.array(invisibles).T, linewidth=0)
       
    
    for e, e_T in zip(E, E_T):
        ax.plot(*e[:,:3].T, color='skyblue', linewidth=3, marker='o', markersize=8)
        ax.plot(*e_T[:,:3].T, color='green', linewidth=3, marker='o', markersize=8)
    
    # visualize motion
    if plot_lines:
        for vecs in zip(V, V_T):
            ax.plot(*np.array(vecs)[:,:3].T, linestyle='--', color='orange')
    
    return ax

In [ ]:
def create_gif(plot, n_frames=20, filename='mygif.gif', fps=20, pause_points=[], pause_start_end=True):
    framenames = []
    for i in range(n_frames + 1):
        # plot chart
        plot(i)

        # create file name and append it to a list
        framename = f'tmp/{i}.png'
        framenames.append(framename)

        # First and last frames stay longer
        if pause_start_end and (i == 0 or i == n_frames) or i in pause_points:
            for i in range(fps):
                framenames.append(framename)

        # save frame
        plt.savefig(framename)
        plt.close()

    # build gif
    with imageio.get_writer(filename, fps=fps) as writer:
        for framename in framenames:
            image = imageio.imread(framename)
            writer.append_data(image)

    # Remove files
    for framename in set(framenames):
        os.remove(framename)

In [ ]:
def show_gif(fname):
    import base64
    from IPython import display
    with open(fname, 'rb') as fd:
        b64 = base64.b64encode(fd.read()).decode('ascii')
    return display.HTML(f'<img src="data:image/gif;base64,{b64}" />')

In [ ]:
I = np.eye(4)
T = np.array([
    [ 1, 0, 0, 0],
    [ 0, 1, 0, 0],
    [ 0, 0, 1, 0],
    [ 1, 1, 0, 1]
])
S = np.array([
    [.5, 0, 0, 0],
    [ 0,.5, 0, 0],
    [ 0, 0,.5, 0],
    [ 0, 0, 0, 1]
])
P = np.array([
    [ 2, -1, -1, 0],
    [-1,  2, -1, 0],
    [-1, -1,  2, 0],
    [ 0,  0,  0, 3]
]) / 3
M = np.array([
    [ 1, -2, -2, 0],
    [-2,  1, -2, 0],
    [-2, -2,  1, 0],
    [ 0,  0,  0, 3]
]) / 3
R = lambda theta : np.array([
    [ np.cos(theta), -np.sin(theta), 0, 0],
    [ np.sin(theta),  np.cos(theta), 0, 0],
    [ 0, 0, 1, 0],
    [ 0, 0, 0, 1]
])

In [ ]:
cube_verts = np.array([
    [0,0,0,1],
    [0,0,1,1],
    [0,1,1,1],
    [0,1,0,1],
    [1,0,0,1],
    [1,0,1,1],
    [1,1,1,1],
    [1,1,0,1],
])
cube_edges = np.array([
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 0),
    (4, 5),
    (5, 6),
    (6, 7),
    (7, 4),
    (0, 4),
    (1, 5),
    (2, 6),
    (3, 7),
])

In [ ]:
ax = vis_transform(cube_verts, cube_edges, R(np.pi/4), plot_lines=False)
ax.set_axis_off()
ax.view_init(30, 10)

In [ ]:
np.expand_dims

# GIFs

In [ ]:
def plot(i):
    ax = vis_transform(cube_verts, cube_edges, R(i / 20 * np.pi/4), plot_lines=False, invisibles=[(-1.4,-1.4,-1.4), (1.5,1.5,1.5)])
    ax.set_axis_off()
    ax.view_init(30, 10)
    
fname = 'rotate.gif'
create_gif(
    plot,
    n_frames=20,
    filename=fname
)
show_gif(fname)

In [ ]:
def plot(i):
    ax = vis_transform(cube_verts, cube_edges, I + i / 20 * (T - I), invisibles=[(0,0,0), (2,2,2)])
    ax.set_axis_off()
    ax.view_init(30, 30)

fname = 'translate.gif'
create_gif(
    plot,
    n_frames=20,
    filename=fname
)
show_gif(fname)

In [ ]:
def plot(i):
    ax = vis_transform(cube_verts, cube_edges, I + i / 20 * (P - I), invisibles=[(-1.4,-1.4,-1.4), (1,1,1)])
    ax.set_axis_off()
    ax.view_init(30, 30)

fname = 'project_mirror.gif'
create_gif(
    plot
    n_frames=40,
    pause_points=[20],
    filename=fname
)
show_gif(fname)

In [ ]:
def plot(i):
    ax = vis_transform(cube_verts, cube_edges, I + 2 * (P - I), invisibles=[(-1.4,-1.4,-1.4), (1,1,1)])
    ax.set_axis_off()
    ax.view_init(30, 5 * i)

fname = 'mirror_spin.gif'
create_gif(
    plot,
    n_frames=72,
    pause_points=[],
    filename=fname,
    fps=10,
    pause_start_end=False
)
show_gif(fname)

In [ ]:
def plot(i):
    ax = vis_transform(cube_verts, cube_edges, I + (i - 20) / 20 * (S - I), invisibles=[(0,0,0), (2,2,2)])
    ax.set_axis_off()
    ax.view_init(30, -60)
    
fname = 'scale.gif'
create_gif(
    plot,
    n_frames=40,
    pause_points=[20],
    filename=fname
)
show_gif(fname)